<a href="https://colab.research.google.com/github/2ovisa/AH2179/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*In this assignment, you will apply regression models to both problems and practice end-to-end model development, parameter tuning, and performance evaluation.*

###Task 1: Bus arrival delays

In [3]:
#Code copied from exercise 1
import pandas as pd

df = pd.read_csv('Exercise2BusData.csv')
df.head(10)

df = df.iloc[:1000]
df = df.drop(['Arrival_time', 'Stop_id', 'Bus_id', 'Line_id'], axis = 1)

#y = the variable we want to predict
#x = the variable that explains what y will be
x = df.drop(['Arrival_delay'], axis=1)
y = df['Arrival_delay']

*Split data into a training and test set*

In [4]:
#Code copied from exercise 1
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

*Create a linear regression model with XGBoost*

- använd parameters for linear boosters, där du ska hitta vilka parameterar som ger bäst resultat. Dessa finns beskrivna här: https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-linear-booster-booster-gblinear

In [32]:
import xgboost
print("XGBoost version:", xgboost.__version__)

from xgboost import XGBRegressor
print("Class being used:", XGBRegressor)

XGBoost version: 3.0.4
Class being used: <class 'xgboost.sklearn.XGBRegressor'>


In [35]:
#https://www.kaggle.com/code/dansbecker/xgboost
import xgboost as xgb
from xgboost import XGBRegressor

my_model = XGBRegressor(n_estimators = 4000, learning_rate=0.02, max_depth = 5, random_state = 42, eval_metric="rmse")

#my_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=5, verbose = False)

my_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])



y_pred = my_model.predict(X_test)

[0]	validation_0-rmse:156.09514
[1]	validation_0-rmse:153.14314
[2]	validation_0-rmse:150.27533
[3]	validation_0-rmse:147.42829
[4]	validation_0-rmse:144.66286
[5]	validation_0-rmse:141.93039
[6]	validation_0-rmse:139.24624
[7]	validation_0-rmse:136.64055
[8]	validation_0-rmse:134.04944
[9]	validation_0-rmse:131.53457
[10]	validation_0-rmse:129.04577
[11]	validation_0-rmse:126.54595
[12]	validation_0-rmse:124.17786
[13]	validation_0-rmse:121.83491
[14]	validation_0-rmse:119.60595
[15]	validation_0-rmse:117.35565
[16]	validation_0-rmse:115.20769
[17]	validation_0-rmse:113.11287
[18]	validation_0-rmse:110.92604
[19]	validation_0-rmse:108.79469
[20]	validation_0-rmse:106.82101
[21]	validation_0-rmse:104.82620
[22]	validation_0-rmse:102.90666
[23]	validation_0-rmse:101.01418
[24]	validation_0-rmse:99.09551
[25]	validation_0-rmse:97.30942
[26]	validation_0-rmse:95.55769
[27]	validation_0-rmse:93.73937
[28]	validation_0-rmse:92.01077
[29]	validation_0-rmse:90.33778
[30]	validation_0-rmse:88.

In [36]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

# DMatrix för xgb.cv
dtrain = xgb.DMatrix(X_train, label=y_train)

params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.02,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_lambda': 1.0,
    'eval_metric': 'rmse'
}

# Kör CV med early stopping
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=4000,
    nfold=5,
    metrics='rmse',
    early_stopping_rounds=50,
    seed=42,
    as_pandas=True
)

best_n = len(cv_results)
print("Optimalt antal boosting-rundor:", best_n)

# Träna en vanlig sklearn-wrapper med det antalet
model = XGBRegressor(
    n_estimators=best_n,
    learning_rate=0.02,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    objective='reg:squarederror',
    verbosity=0
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("R² på test:", r2_score(y_test, y_pred))

Optimalt antal boosting-rundor: 705
R² på test: 0.9898396730422974


*Evaluation*

In [26]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Absolute Error: 11.318708419799805
Mean Squared Error: 248.5261688232422
R-squared: 0.99000483751297


r _squared utan förändring =
0.989790678024292

med n_estimators = 1000, learning_rate = 0.05 = 0.9898112416267395

n_estimators = 500: = 0.9903118014335632

n = 100, 0.9908506870269775
n = 150, 0.9910303354263306
n = 160, 0.991042971611023
n = 165, 0.9910459518432617

*AI model training techniques*

*Normalization*

In [7]:
#Code copied from exercise 1
from sklearn.preprocessing import StandardScaler

# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

# Define a parameter grid for hyperparameter tuning
param_grid = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10],
    'epsilon': [0.01,0.1,1, 10]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(SVR(), param_grid, cv=5, verbose=2)

# Fit the grid search to the scaled training data
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters
best_params = grid_search.best_params_

print("Best Parameters:", best_params)
print("Best Score:", grid_search.best_score_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END .................C=0.1, epsilon=0.01, kernel=linear; total time=   0.1s
[CV] END .................C=0.1, epsilon=0.01, kernel=linear; total time=   0.0s
[CV] END .................C=0.1, epsilon=0.01, kernel=linear; total time=   0.0s
[CV] END .................C=0.1, epsilon=0.01, kernel=linear; total time=   0.0s
[CV] END .................C=0.1, epsilon=0.01, kernel=linear; total time=   0.0s
[CV] END ....................C=0.1, epsilon=0.01, kernel=rbf; total time=   0.0s
[CV] END ....................C=0.1, epsilon=0.01, kernel=rbf; total time=   0.0s
[CV] END ....................C=0.1, epsilon=0.01, kernel=rbf; total time=   0.0s
[CV] END ....................C=0.1, epsilon=0.01, kernel=rbf; total time=   0.0s
[CV] END ....................C=0.1, epsilon=0.01, kernel=rbf; total time=   0.1s
[CV] END ..................C=0.1, epsilon=0.1, kernel=linear; total time=   0.0s
[CV] END ..................C=0.1, epsilon=0.1, 

*Train a SVM regression model*

In [13]:
# Create an SVR model with the best parameters from the grid search
best_svr = SVR(kernel=best_params['kernel'], C=best_params['C'], epsilon=best_params['epsilon'])
best_svr.fit(X_train, y_train)

SVR(C=10, epsilon=10, kernel='linear')

In [14]:
y_pred = best_svr.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Absolute Error: 9.029471488864578
Mean Squared Error: 167.1210201664315
R-squared: 0.993278787107813
